# Projeto 1
---

* **Alunos**: 

    1. Bernardo Tameirão (**12733212**) 
    2. João Gabriel Sasseron Roberto Amorim (**12542564**)

* **Disciplina**: SCC0250 - Computação Gráfica (2024/2)

* **Docente**: Jean Roberto Ponciano

# Configurações Iniciais 

---

In [1]:
import glfw
from OpenGL.GL import *
import OpenGL.GL.shaders
import numpy as np
import glm 
import math
from PIL import Image

In [2]:
# Inicialização da janela
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE)

# Configurações da janela
WIDTH  = 1600
HEIGHT = 1200

window = glfw.create_window(WIDTH, HEIGHT, "Medieval", None, None)
glfw.make_context_current(window)

# Interação com o Vertex Shaders do Pipeline Gráfico
vertex_code = """
            attribute vec3 position;
            uniform mat4 model;
            uniform mat4 view;
            uniform mat4 projection;     
            
            attribute vec2 texture_coord;
            varying vec2 coordinates_texture;   
            
            void main(){
                gl_Position = projection * view * model * vec4(position,1.0);
                coordinates_texture = vec2(texture_coord);
            }
        """
        
# Interação com o Fragment Shaders do Pipeline Gráfico
fragment_code = """
            uniform vec4 color;
            varying vec2 coordinates_texture;
            uniform sampler2D imagem;
            void main(){
                vec4 texture = texture2D(imagem, coordinates_texture);
                gl_FragColor = texture;
            }
        """

# Requisitando slots para a GPU para os programas acima
# Request a program and shader slots from GPU
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)

# Associação do código fonte aos slots solicitados
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

# Compila o Vertex Shader
glCompileShader(vertex)

# Em caso de erros encerra o programa
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")

# Compila o Fragment Shader
glCompileShader(fragment)

# Em caso de erros encerra o programa
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

# Associação dos programas compilados ao programa principal
glAttachShader(program, vertex)
glAttachShader(program, fragment)

# Link dos programas
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
# Definição de nosso programa como padrão
glUseProgram(program)

# Funções úteis

---

In [3]:
def model(tx, ty, tz, rx, ry, rz, sx, sy, sz, angle=0.0):
    """
    Gera uma matriz de transformação composta aplicando translação, rotação (em X, Y, Z)
    e escala, nesta ordem.

    Args:
        tx, ty, tz: Valores de translação ao longo dos eixos X, Y e Z.
        rx, ry, rz: Ângulos de rotação (em graus) ao longo dos eixos X, Y e Z.
        sx, sy, sz: Fatores de escala ao longo dos eixos X, Y e Z.
        angle: Ângulo de rotação 

    Returns:
        matrix_transform: Matriz 4x4 resultante após aplicar translação, rotação e escala.
    """
    # Inicializa a matriz de transformação como a matriz identidade
    matrix_transform = matrix_transform = glm.mat4(1.0)
    
    # Aplica a translação
    matrix_transform = glm.translate(matrix_transform, glm.vec3(tx, ty, tz))
    
    # Aplica rotação
    if angle != 0:
        matrix_transform = glm.rotate(matrix_transform, 
                                      angle, 
                                      glm.vec3(rx, ry, rz))
        
    # Aplica a escala
    matrix_transform = glm.scale(matrix_transform, glm.vec3(sx, sy, sz))
        
    # Tranforma em array
    matrix_transform = np.array(matrix_transform)
    
    # Retorna a matriz de transformação final
    return matrix_transform

In [4]:
def view(camera_position, camera_front, camera_up):
    """
    Cria uma matriz de visualização (view matrix) para ajustar a posição e a direção da câmera na cena 3D.

    Args:
        camera_position: A posição da câmera no espaço 3D, representada como um vetor 3D.
        camera_front: A direção para a qual a câmera está apontando, também representada como um vetor 3D.
        camera_up: O vetor que define a orientação "para cima" da câmera.

    Returns:
        view: A matriz de visualização (view matrix) convertida para um array NumPy. 
    """
    view = glm.lookAt(camera_position, camera_position + camera_front, camera_up);
    view = np.array(view)
    
    return view

In [5]:
def projection():
    """
    Cria uma matriz de projeção em perspectiva para definir o campo de visão da câmera no espaço 3D.

    Returns:
        projection: A matriz de projeção (projection matrix) convertida para um array NumPy.
    """
    projection = glm.perspective(glm.radians(45.0), WIDTH / HEIGHT, 0.1, 1000.0)
    projection = np.array(projection)    
    
    return projection

# Leitura dos modelos

---

In [6]:
def load_model_from_file(filename):
    """Loads a Wavefront OBJ file. """
    objects = {}
    vertices = []
    texture_coords = []
    faces = []

    material = None

    # abre o arquivo obj para leitura
    for line in open(filename, "r"): ## para cada linha do arquivo .obj
        if line.startswith('#'): continue ## ignora comentarios
        values = line.split() # quebra a linha por espaço
        if not values: continue


        ### recuperando vertices
        if values[0] == 'v':
            vertices.append(values[1:4])


        ### recuperando coordenadas de textura
        elif values[0] == 'vt':
            texture_coords.append(values[1:3])

        ### recuperando faces 
        elif values[0] in ('usemtl', 'usemat'):
            material = values[1]
        elif values[0] == 'f':
            face = []
            face_texture = []
            for v in values[1:]:
                w = v.split('/')
                face.append(int(w[0]))
                if len(w) >= 2 and len(w[1]) > 0:
                    face_texture.append(int(w[1]))
                else:
                    face_texture.append(0)

            faces.append((face, face_texture, material))

    model = {}
    model['vertices'] = vertices
    model['texture'] = texture_coords
    model['faces'] = faces

    return model

In [7]:
# glEnable(GL_TEXTURE_2D)
# qtd_texturas = 1
# textures = glGenTextures(qtd_texturas)

def load_texture_from_file(texture_id, img_path):
    glBindTexture(GL_TEXTURE_2D, texture_id)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_S, GL_REPEAT) #S é a coordenada U da textura
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_T, GL_REPEAT) #T é a coordenada V da textura
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR)
    img = Image.open(img_path)
    img_width = img.size[0]
    img_height = img.size[1]
    image_data = img.tobytes("raw", "RGB", 0, -1)

    glTexImage2D(GL_TEXTURE_2D, 0, GL_RGB, img_width, img_height, 0, GL_RGB, GL_UNSIGNED_BYTE, image_data)

# Dados que serão utilizados

---

In [8]:

# Leitura dos modelos
model_house   = load_model_from_file('objects\\house\\snow.obj')
model_shield  = load_model_from_file('objects\\shield\\shield.obj')
model_anvil   = load_model_from_file('objects\\anvil\\anvil.obj')
model_spears  = load_model_from_file('objects\\spears\\spears.obj')
model_ground  = load_model_from_file('objects\\enviroment\\ground.obj')
model_sky  = load_model_from_file('objects\\enviroment\\sky.obj')
model_tree  = load_model_from_file('objects\\tree\\tree.obj')
model_monster  = load_model_from_file('objects\\monster\\monster.obj')

# Listas auxiliares que serão armazenadas as coordenadas
vertices_list = []    
textures_coord_list = []

# Controle da posições dos vértices
set_position = []   

# Inserindo vertices do modelo no vetor de vertices
for model_aux in [model_house, model_shield, model_anvil, 
                  model_spears, model_ground, model_sky,
                  model_tree, model_monster]:
    start = len(vertices_list)
    
    for face in model_aux['faces']:
        for vertice_id in face[0]: 
            vertices_list.append(model_aux['vertices'][vertice_id-1] )
        
        for texture_id in face[1]:
            textures_coord_list.append(model_aux['texture'][texture_id - 1] )
    end = len(vertices_list)
    
    set_position.append((start, end - start))

# Carregando as texturas no buffer
load_texture_from_file(0,'objects\\house\\snow.jpg')
load_texture_from_file(1,'objects\\shield\\shield.jpg')
load_texture_from_file(2,'objects\\anvil\\anvil.png')
load_texture_from_file(3,'objects\\spears\\spears.png')
load_texture_from_file(4,'objects\\enviroment\\ground.jpg')
load_texture_from_file(5,'objects\\enviroment\\sky.png')
load_texture_from_file(6,'objects\\tree\\tree.jpg')
load_texture_from_file(7,'objects\\monster\\monster.png')

In [9]:
# Requisitando espaço na GPU
buffer = glGenBuffers(2)

##### Vértices
# Transformando os dados para serem colocados na GPU
vertices = np.zeros(len(vertices_list), [("position", np.float32, 3)])
vertices['position'] = vertices_list

# Inserção dos dados
glBindBuffer(GL_ARRAY_BUFFER, buffer[0])
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_STATIC_DRAW)
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)
loc_vertices = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc_vertices)
glVertexAttribPointer(loc_vertices, 3, GL_FLOAT, False, stride, offset)

##### Textura
# Transformando a textura para a GPU
textures = np.zeros(len(textures_coord_list), [("position", np.float32, 2)]) # duas coordenadas
textures['position'] = textures_coord_list

# Inserção dos dados
glBindBuffer(GL_ARRAY_BUFFER, buffer[1])
glBufferData(GL_ARRAY_BUFFER, textures.nbytes, textures, GL_STATIC_DRAW)
stride = textures.strides[0]
offset = ctypes.c_void_p(0)
loc_texture_coord = glGetAttribLocation(program, "texture_coord")
glEnableVertexAttribArray(loc_texture_coord)
glVertexAttribPointer(loc_texture_coord, 2, GL_FLOAT, False, stride, offset)

# Objetos

---

Agora, vamos desenhar os objetos que irão compor nossa cena. Para isso, criaremos uma classe genérica que conterá as funções básicas que serão reutilizadas, além de classes específicas para cada objeto, que irão herdar e especializar as funcionalidades da classe genérica.

In [10]:

class Object3D:
    """
    Representa um objeto 3D com transformações de translação, rotação e escala. 
    A classe permite definir e atualizar as coordenadas de transformação e controlar 
    a rotação e a seleção do objeto.
    """
    def __init__(self, tx, ty, tz, rx, ry, rz, sx, sy, sz, angle, index):
        self.tx, self.ty, self.tz = tx, ty, tz # Translação inicial
        self.rx, self.ry, self.rz = rx, ry, rz # Rotação inicial
        self.sx, self.sy, self.sz = sx, sy, sz # Escala inicial
        self.rotate = True  # Controla se a rotação está habilitada
        self.select = 0     # Identificador de seleção do objeto
        self.angle = angle  # Ângulo padrão para rotação
        self.index = index

    def set_coordinates(self, tx, ty, tz, rx, ry, rz, sx, sy, sz):
        """
        Define as coordenadas do objeto 3D.

        Parâmetros:
        tx, ty, tz: Valores de translação ao longo dos eixos X, Y e Z.
        rx, ry, rz: Ângulos de rotação ao longo dos eixos X, Y e Z.
        sx, sy, sz: Fatores de escala ao longo dos eixos X, Y e Z.
        """
        self.tx, self.ty, self.tz = tx, ty, tz
        self.rx, self.ry, self.rz = rx, ry, rz
        self.sx, self.sy, self.sz = sx, sy, sz

    def update_translate(self, tx, ty, tz):
        """
        Atualiza a translação do objeto somando os valores fornecidos aos valores atuais.

        Parâmetros:
        tx, ty, tz: Incrementos de translação ao longo dos eixos X, Y e Z.
        """
        self.tx += tx
        self.ty += ty
        self.tz += tz

    def update_rotate(self, rx, ry, rz):
        """
        Atualiza os ângulos de rotação se a rotação estiver habilitada.

        Parâmetros:
        rx, ry, rz: Incrementos dos ângulos de rotação ao longo dos eixos X, Y e Z.
        """
        if self.rotate:
            self.rx += rx
            self.ry += ry
            self.rz += rz

    def update_scale(self, sx, sy, sz):
        """
        Atualiza a escala do objeto somando os valores fornecidos aos fatores de escala atuais.

        Parâmetros:
        sx, sy, sz: Incrementos dos fatores de escala ao longo dos eixos X, Y e Z.
        """
        self.sx += sx
        self.sy += sy
        self.sz += sz

    def set_select(self, select):
        """
        Define o identificador de seleção do objeto.

        Parâmetros:
        select: Identificador numérico para o objeto.
        """
        self.select = select
        
    def __str__(self):
        return f'''
        translation: {self.tx}, {self.ty}, {self.tz}\n
        rotation:    {self.rx}, {self.ry}, {self.rz}\n
        scale:       {self.sx}, {self.sy}, {self.sz}\n
        angle: {self.angle}
    '''
    
    def draw_object(self):
        mat_model = model(self.tx, self.ty, self.tz, 
                          self.rx, self.ry, self.rz, 
                          self.sx, self.sy, self.sz,
                          self.angle)
        
        loc_model = glGetUniformLocation(program, "model")

        glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)

        glBindTexture(GL_TEXTURE_2D, self.index)
        
        glDrawArrays(GL_TRIANGLES, set_position[self.index][0], set_position[self.index][1])

In [11]:

import random
import math

def coordenadas_aleatorias(min_ang, max_ang, min_lim, max_lim):
    # Gera um ângulo aleatório entre 0 e 360 graus
    angulo = random.uniform(min_ang, max_ang)
    # Gera uma distância aleatória entre 5 e 20 unidades
    distancia = random.uniform(min_lim, max_lim)
    # Calcula tx e tz usando trigonometria
    tx = distancia * math.cos(math.radians(angulo))
    tz = distancia * math.sin(math.radians(angulo))
    return tx, tz

In [12]:
house = Object3D(tx=0.00, ty=0.00, tz=0.0,
                 rx=0.00, ry=0.00, rz=0.0,
                 sx=0.05, sy=0.05, sz=0.05,
                 angle=0.0, index=0)

shield = Object3D(tx=1.20, ty=0.03  , tz=2.01,
                  rx=0.00, ry=473708, rz=0.00,
                  sx=0.02, sy=0.02  , sz=0.02,
                  angle=92, index=1)

anvil = Object3D(tx=-1.22, ty=0.00   , tz=0.00,
                  rx=0.00, ry=10262.9, rz=0.00,
                  sx=0.15, sy=0.15  ,  sz=0.15,
                  angle=1.65, index=2)

spears = Object3D(tx=1.21, ty=0.03    , tz=-0.03,
                  rx=0.00,  ry=23651.25, rz=0.00,
                  sx=1.15,  sy=1.15    , sz=1.15,
                  angle=3.15, index=3)

ground = Object3D(tx=0.00, ty=-0.16, tz=0.00,
                  rx=0.00, ry=0.00 , rz=0.00,
                  sx=1.00, sy=1.00 , sz=1.00,
                  angle=0, index=4)

sky = Object3D(tx=0.00, ty=0.00, tz=0.00,
               rx=0.00, ry=0.00, rz=0.00,
               sx=1.00, sy=1.00, sz=1.00,
               angle=0, index=5)

tree_list = []

for i in range(60):
    tx, tz = coordenadas_aleatorias(0, 360, 15, 45)
    
    tree = Object3D(tx=tx, ty=0.00, tz=tz    ,
                    rx=0.00, ry=0.00, rz=0.00,
                    sx=0.40, sy=0.40, sz=0.40,
                    angle=0, index=6)
    
    tree_list.append(tree)

monster_list = []

for i in range(3):
    tx, tz = coordenadas_aleatorias(180, 270, 10, 15)
    
    monster = Object3D(tx=tx  , ty=0.00, tz=tz  ,
                       rx=0.00, ry=0.00, rz=0.0 ,
                       sx=0.40, sy=0.40, sz=0.40,
                       angle=0.0, index=7)
    
    monster_list.append(monster)

# Janela

---

Por fim, iremos criar os objetos, as possíveis interações e suas visuzalização.

In [13]:
# Mostrando as malhas
wireframe_mode = False

# Defini as configurações da câmera
class Camera():
    def __init__(self):
        self.camera_speed    = 0.5
        self.camera_position = glm.vec3(0.0)
        self.camera_front    = glm.vec3(0.0,  0.0, -1.0)
        self.camera_up       = glm.vec3(0.0,  1.0,  0.0)

camera = Camera()

# Definif as configurações do mouse
class Mouse():
    def __init__(self):
        self.first_mouse = True
        self.yaw   = -90.0 
        self.pitch = 0.0
        self.lastX = HEIGHT / 2
        self.lastY = WIDTH / 2

mouse = Mouse()

# Objetos selecionados
select_obj = None

In [14]:
def key_event(window, key, scancode, action, mods):
    global select_obj, camera, wireframe_mode  # Referencia as variáveis globais usadas
    
    if action == glfw.PRESS:
        # Seleção de objetos 3D com as teclas de 1 a 5 (clubs, hearths, diamonds, spades, joker)
        if key == glfw.KEY_1:
            select_obj = house
        elif key == glfw.KEY_2:
            select_obj = shield
        elif key == glfw.KEY_3:
            select_obj = anvil
        elif key == glfw.KEY_4:
            select_obj = spears
        elif key == glfw.KEY_5:
            select_obj = ground
        elif key == glfw.KEY_6:
            select_obj = tree
            
    # Se um objeto foi selecionado, ele é marcado como selecionado (set_select(1)),
        # e os demais objetos têm sua seleção removida (set_select(0)).
        if select_obj != None:
            select_obj.set_select(1)
            for obj in [house, shield]:
                if obj != select_obj:
                    obj.set_select(0)
    
    if action == glfw.PRESS or action == glfw.REPEAT:
        # Movimentação (translação) do objeto com as teclas W, A, S, D
        if key == glfw.KEY_W:  
            camera.camera_position += camera.camera_speed * camera.camera_front 
        elif key == glfw.KEY_S:
            camera.camera_position -= camera.camera_speed * camera.camera_front 
        elif key == glfw.KEY_A:
            camera.camera_position -= glm.normalize(glm.cross(camera.camera_front, camera.camera_up)) * camera.camera_speed
        elif key == glfw.KEY_D:  
            camera.camera_position += glm.normalize(glm.cross(camera.camera_front, camera.camera_up)) * camera.camera_speed
        
        # Tecla 'P' para alternar entre modo wireframe (linhas) e modo de preenchimento (sólido)
        if key == glfw.KEY_P:
            if wireframe_mode:
                glPolygonMode(GL_FRONT_AND_BACK, GL_LINE)  # Ativa wireframe
            else:
                glPolygonMode(GL_FRONT_AND_BACK, GL_FILL)  # Ativa modo sólido
            
            wireframe_mode = not wireframe_mode  # Alterna o estado do modo wireframe

    # Reseta valores de translação, rotação e escala
    tx, ty, tz = 0.0, 0.0, 0.0
    rx, ry, rz = 0.0, 0.0, 0.0
    sx, sy, sz = 0.0, 0.0, 0.0
    
    if action == glfw.PRESS or action == glfw.REPEAT:
        # Movimentação (translação) do objeto com as teclas W, A, S, D
        if key == glfw.KEY_Y:  
            ty = +0.01  # Move para cima
        elif key == glfw.KEY_H:
            ty = -0.01  # Move para baixo
        elif key == glfw.KEY_U:
            tx = -0.01  # Move para a esquerda
        elif key == glfw.KEY_J:  
            tx = +0.01  # Move para a direita
        elif key == glfw.KEY_I:
            tz = -0.01  # Move para a esquerda
        elif key == glfw.KEY_K:  
            tz = +0.01  # Move para a direita
        
        # Escala do objeto com as setas UP/DOWN (aumentar/diminuir Y) e LEFT/RIGHT (aumentar/diminuir X)
        if select_obj != None:
            if key == glfw.KEY_Z:  
                select_obj.ry = not select_obj.ry  # Aumenta o eixo Y
            elif key == glfw.KEY_X:
                select_obj.rx = not select_obj.rx  # Aumenta o eixo Y
            elif key == glfw.KEY_C:
                select_obj.rz = not select_obj.rz  # Aumenta o eixo Y
                
            # Rotação do objeto com as teclas PAGE_UP/PAGE_DOWN (altera o ângulo de rotação)
            if key == glfw.KEY_PAGE_UP:  
                select_obj.angle += 0.1  # Rotaciona no sentido anti-horário no eixo Y
            elif key == glfw.KEY_PAGE_DOWN:
                select_obj.angle -= 0.1  # Rotaciona no sentido horário no eixo Y
            
            select_obj.update_translate(tx=tx, ty=ty, tz=tz)  # Aplica a translação
            select_obj.update_scale(sx=sx, sy=sy, sz=0.0)  # Aplica a escala

glfw.set_key_callback(window,key_event)

def mouse_event(window, xpos, ypos):
    global camera, mouse
    
    if mouse.first_mouse:
        mouse.lastX = xpos
        mouse.lastY = ypos
        mouse.first_mouse = False

    xoffset = xpos - mouse.lastX
    yoffset = mouse.lastY - ypos
    mouse.lastX = xpos
    mouse.lastY = ypos

    sensitivity = 0.3 
    xoffset *= sensitivity
    yoffset *= sensitivity

    mouse.yaw += xoffset;
    mouse.pitch += yoffset;

    if mouse.pitch >= 90.0: 
        mouse.pitch = 90.0
    
    if mouse.pitch <= -90.0: 
        mouse.pitch = -90.0

    front = glm.vec3()
    front.x = math.cos(glm.radians(mouse.yaw)) * math.cos(glm.radians(mouse.pitch))
    front.y = math.sin(glm.radians(mouse.pitch))
    front.z = math.sin(glm.radians(mouse.yaw)) * math.cos(glm.radians(mouse.pitch))
    camera.camera_front = glm.normalize(front)
    
glfw.set_cursor_pos_callback(window, mouse_event)

In [15]:
glfw.show_window(window)
glfw.set_cursor_pos(window, mouse.lastX, mouse.lastY)

In [16]:
glEnable(GL_DEPTH_TEST)

while not glfw.window_should_close(window):
    glfw.poll_events() 
    
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)    
    glClearColor(1.0, 1.0, 1.0, 1.0)
    
    # Itera sobre a lista de objetos a serem desenhados
    for obj in [house, shield, anvil, spears, ground, sky] + monster_list + tree_list:
        obj.draw_object()
    
    mat_view = view(camera.camera_position, 
                    camera.camera_front, 
                    camera.camera_up)
    
    loc_view = glGetUniformLocation(program, "view")
    glUniformMatrix4fv(loc_view, 1, GL_TRUE, mat_view)

    mat_projection = projection()
    loc_projection = glGetUniformLocation(program, "projection")
    glUniformMatrix4fv(loc_projection, 1, GL_TRUE, mat_projection)   
    
    glfw.swap_buffers(window)


glfw.terminate()

In [17]:
print(tree)


        translation: 17.809264657685453, 0.0, -4.898876995031425

        rotation:    0.0, 0.0, 0.0

        scale:       0.4, 0.4, 0.4

        angle: 0
    
